## Set up network and install software

In [ ]:
from state import *
from IPython.core.getipython import get_ipython

# Import libraries
%run ../setup/include/include_libraries.py
# Import selected slice name
%run ../slice_info/selected_slice.py


# Import topology_variables

# Use the button in the GUI automatically does it for you
if os.getenv('SELECTED_SLICE') is not None:
    SELECTED_SLICE = os.getenv('SELECTED_SLICE')

# If you manually run the notebook, please specify the slice name in the line below
# SELECTED_SLICE = 'slice_name'

path = f'../slice_info/{SELECTED_SLICE}/topology_variables.ipynb'
get_ipython().run_line_magic('run', path)

import write_value


In [ ]:
try:
    slice = fablib.get_slice(name=selected_slice)
    available_ips = list(subnet)[1:]
    #node1
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network_name)
    if (node1_addr == ''):
        node1_addr = available_ips.pop(0)
        write_value.write_value_to_file(path, 'node1_addr', node1_addr)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_device_name()}')
    
    
    #node2
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network_name)
    if (node2_addr == ''):
        node2_addr = available_ips.pop(0)
        write_value.write_value_to_file(path, 'node2_addr', node2_addr)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=subnet)
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_device_name()}')
    
    success = True
except Exception as e:
    print(f"Exception: {e}")
    success = False
    raise SystemExit("Stopping notebook execution due to exception")

## Update state

In [ ]:
if success:
    update_state_local(selected_slice, 'SLICE_CONFIGURED')
    update_state_in_fim(selected_slice, read_state_from_local(selected_slice))
else:
    print ('State is not updated due to error.')